In [1]:
import os
import cv2
from glob import glob

In [7]:
def yolo_to_cascade(yolo_dir, image_dir, output_file):
    
    supported_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff", "*.webp"]
    image_files = []
    for ext in supported_extensions:
        image_files.extend(glob(os.path.join(image_dir, ext)))
    image_files.sort()

    yolo_files = glob(os.path.join(yolo_dir, "*.txt"))
    yolo_files.sort()

    if len(image_files) != len(yolo_files):
        print("Greška: Broj slika i YOLO anotacija nije jednak!")
        return

    cascade_annotations = []

    for image_path, yolo_path in zip(image_files, yolo_files):
        image = cv2.imread(image_path)
        img_height, img_width, _ = image.shape

        with open(yolo_path, 'r') as f:
            lines = f.readlines()

        objects = []
        for line in lines:
            parts = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, parts)

            x_center = int(x_center * img_width)
            y_center = int(y_center * img_height)
            width = int(width * img_width)
            height = int(height * img_height)

            x1 = x_center - width // 2
            y1 = y_center - height // 2

            objects.append(f"{x1} {y1} {width} {height}")

        if objects:
            cascade_annotations.append(f"{image_path} {len(objects)} " + " ".join(objects))

    with open(output_file, 'w') as f:
        f.write("\n".join(cascade_annotations))


In [15]:
#knife images
yolo_to_cascade("knife_lables", "knife_pos_img", "knife_pos.txt")

#pistol images
yolo_to_cascade("pistol_lables", "pistol_pos_img", "pistol_pos.txt")

#bill images
yolo_to_cascade("bill_lables", "bill_pos_img", "bill_pos.txt")

#card images
yolo_to_cascade("card_lables", "card_pos_img", "card_pos.txt")

In [9]:
def generate_negative_description_file():
    
    with open('neg.txt', 'w') as f:
        for filename in os.listdir('neg_img'):
            f.write('neg_img/' + filename + '\n')

In [11]:
generate_negative_description_file()

In [3]:
def evaluate_classifier(cascade_path, test_images_dir, annotations_dir):
  
    cascade = cv2.CascadeClassifier(cascade_path)
    if cascade.empty():
        raise FileNotFoundError(f"Haar klasifikator nije pronađen na {cascade_path}")
    
    true_positive = 0
    false_positive = 0
    false_negative = 0

    for image_name in os.listdir(test_images_dir):
        image_path = os.path.join(test_images_dir, image_name)
        annotation_path = os.path.join(annotations_dir, os.path.splitext(image_name)[0] + ".txt")

        image = cv2.imread(image_path)
        h, w, _ = image.shape

        with open(annotation_path, 'r') as file:
            true_boxes = []
            for line in file:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                x_center, y_center, width, height = x_center * w, y_center * h, width * w, height * h
                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)
                true_boxes.append((x1, y1, x2, y2))

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        detected_boxes = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        matched = set()
        for (tx1, ty1, tx2, ty2) in true_boxes:
            match_found = False
            for (dx, dy, dw, dh) in detected_boxes:
                dx1, dy1, dx2, dy2 = dx, dy, dx + dw, dy + dh
                iou = calculate_iou((tx1, ty1, tx2, ty2), (dx1, dy1, dx2, dy2))
                if iou >= 0.5:
                    true_positive += 1
                    matched.add((dx1, dy1, dx2, dy2))
                    match_found = True
                    break
            if not match_found:
                false_negative += 1

        # Detekcije koje nisu u anotacijama su lažno pozitivne
        for detected_box in detected_boxes:
            dx1, dy1, dx2, dy2 = detected_box[0], detected_box[1], detected_box[0] + detected_box[2], detected_box[1] + detected_box[3]
            if (dx1, dy1, dx2, dy2) not in matched:
                false_positive += 1

    # Izračunavanje metrika
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score


def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union = area1 + area2 - intersection
    return intersection / union if union > 0 else 0


In [7]:
precision, recall, f1 = evaluate_classifier(
    cascade_path="cascades/cascade_card/cascade.xml",
    test_images_dir="test/card/img",
    annotations_dir="test/card/label"
)
print('Klasa Card')
print(f"Preciznost: {precision:.3f}, Odziv: {recall:.3f}, F1 skor: {f1:.3f}")

Klasa Card
Preciznost: 0.036, Odziv: 0.193, F1 skor: 0.060


In [13]:
precision, recall, f1 = evaluate_classifier(
    cascade_path="cascades/cascade_bill/cascade.xml",
    test_images_dir="test/bill/img",
    annotations_dir="test/bill/label"
)
print('Klasa Bill')
print(f"Preciznost: {precision:.3f}, Odziv: {recall:.3f}, F1 skor: {f1:.3f}")

Klasa Bill
Preciznost: 0.048, Odziv: 0.245, F1 skor: 0.080


In [15]:
precision, recall, f1 = evaluate_classifier(
    cascade_path="cascades/cascade_knife/cascade.xml",
    test_images_dir="test/knife/img",
    annotations_dir="test/knife/label"
)
print('Klasa Knife')
print(f"Preciznost: {precision:.3f}, Odziv: {recall:.3f}, F1 skor: {f1:.3f}")

Klasa Knife
Preciznost: 0.007, Odziv: 0.087, F1 skor: 0.013


In [17]:
precision, recall, f1 = evaluate_classifier(
    cascade_path="cascades/cascade_pistol/cascade.xml",
    test_images_dir="test/pistol/img",
    annotations_dir="test/pistol/label"
)
print('Klasa Pistol')
print(f"Preciznost: {precision:.3f}, Odziv: {recall:.3f}, F1 skor: {f1:.3f}")

Klasa Pistol
Preciznost: 0.114, Odziv: 0.472, F1 skor: 0.184


In [5]:
def visualize_detections(cascade_path, image_folder):
   
    cascade = cv2.CascadeClassifier(cascade_path)
    if cascade.empty():
        raise FileNotFoundError(f"Haar klasifikator nije pronađen na {cascade_path}")

    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
            
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        detections = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in detections:
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
        cv2.imshow('Detections', image)

        key = cv2.waitKey(0)  
        if key == ord('q'):   
            break
    
    cv2.destroyAllWindows()

In [7]:
visualize_detections("cascades/cascade_card/cascade.xml", "test/card/img")

In [9]:
visualize_detections("cascades/cascade_bill/cascade.xml", "test/bill/img")

In [11]:
visualize_detections("cascades/cascade_knife/cascade.xml", "test/knife/img")

In [13]:
visualize_detections("cascades/cascade_pistol/cascade.xml", "test/pistol/img")